<span style="font-size: 5em">🦜</span>

# __LangGraph Essentials__
# Build A Workflow
<div style="display:flex; align-items:flex-start;">
  <img src="assets/EmailWorkflow.png" width="600" style="margin-right:15px;"/>
</div>

## Setup

Load and/or check for needed environmental variables

In [ ]:
from dotenv import load_dotenv, find_dotenv
from env_utils import doublecheck_env

path = find_dotenv()
print("Loaded env from:", path)

# Load environment variables from .env
load_dotenv(find_dotenv())

# Check and print results
doublecheck_env(path)

In [ ]:
import uuid
import os
from typing import Literal, TypedDict
from IPython.display import Image, display

# Define state schemas

In [ ]:
class EmailClassification(TypedDict):
    intent: Literal["question", "bug", "billing", "feature", "complex"]
    urgency: Literal["low", "medium", "high", "critical"]
    topic: str
    summary: str

class EmailAgentState(TypedDict):
    # Raw email data
    email_content: str
    sender_email: str
    email_id: str

    # Classification result
    classification: EmailClassification | None

    # Bug tracking
    ticket_id: str | None

    # Raw search results
    search_results: list[str] | None
    customer_history: dict | None

    # Generated content
    draft_response: str | None

# Define Nodes, Edges

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.types import Command, interrupt
from langgraph.graph import END, START, StateGraph

def read_email(state: EmailAgentState) -> EmailAgentState:
    """Extract and parse email content"""
    pass

llm = ChatOpenAI(model="gpt-5-mini")

def classify_intent(state: EmailAgentState) -> EmailAgentState:
    """Use LLM to classify email intent and urgency, then route accordingly"""

    # Create structured LLM that returns EmailClassification dict
    structured_llm = llm.with_structured_output(EmailClassification)

    classification_prompt = f"""
    Analyze this customer email and classify it:

    Email: {state['email_content']}
    From: {state['sender_email']}

    Provide classification, including intent, urgency, topic, and summary
    """

    # Get structured response directly as a dict
    classification = structured_llm.invoke(classification_prompt)

    # Store classification as a single dict in state
    return {"classification": classification}

def search_documentation(state: EmailAgentState) -> EmailAgentState:
    """Search knowledge base for relevant information"""

    # Build search query from classification
    classification = state.get('classification', {})
    query = f"{classification.get('intent', '')} {classification.get('topic', '')}"

    try:
        # Implement search logic here
        search_results = [
            "--Search_result_1--",
            "--Search_result_2--",
            "--Search_result_3--"
        ]
    except SearchAPIError as e:
        # For recoverable search errors, store error and continue
        search_results = [f"Search temporarily unavailable: {str[e]}"]

    return {"search_results": search_results} # Raw search results or error

def bug_tracking(state: EmailAgentState) -> EmailAgentState:
    """Create or update bug tracking ticket"""

    # Create ticket in your bug tracking system
    ticket_id = f"BUG_{uuid.uuid4()}"

    return {"ticket_id": ticket_id}

def write_response(state: EmailAgentState) -> Command[Literal["human_review", "send_reply"]]:
    "Generate response using context and route based on quality"""

    classification = state.get('classification', {})

    # Format context from raw state data on demand
    context_sections = []

    if state.get('search_results'):
        # Format search results for the prompt
        formatted_docs = "\n".join([f"- {doc}" for doc in state['search_results']])
        context_sections.append(f"Relevant documentation:\n{formatted_docs}")

    if state.get('customer_history'):
        # Format customer data for the prompt
        context_sections.append(f"Customer tier: {state['customer_history'].get('tier', 'standard')}")

    # Build the prompt with formatted context
    draft_prompt = f"""
    Draft a response to this customer email:
    {state['email_content']}

    Email intent: {classification.get('intent', 'unkown')}
    Urgency level: {classification.get('urgency', 'medium')}

    {chr(10).join(context_sections)}

    Guidelines:
    - Be professional and helpful
    - Address their specific concern
    - Use the provided documentation when relevant
    - Be brief
    """

    response = llm.invoke(draft_prompt)

    # Determine if human review is needed based on urgency and intent
    needs_review = (
        classification.get('urgency') in ['high', 'critical'] or
        classification.get('intent') == 'complex'
    )

    # Route to the appropriate next node
    if needs_review:
        goto = "human_review"
        print("Needs approval")
    else:
        goto = "send_reply"

    return Command(
        update = {"draft_response": response.content},
        goto = goto
    )

def human_review(state: EmailAgentState) -> Command[Literal["send_reply", END]]:
    """Pause for human review using interrupt and route based on decision"""

    classification = state.get('classification', {})

    # Interrupt() must come first - any code before it will re-run on resume
    human_decision = interrupt({
        "email_id": state['email_id'],
        "original_email": state['email_content'],
        "draft_response": state.get('draft_response', ""),
        "urgency": classification.get('urgency'),
        "intent": classification.get('intent'),
        "action": "Please review and approve/edit this response"
    })

    # Now process the human's decision
    if human_decision.get("approved"):
        return Command(
            update = {"draft_response": human_decision.get("edited_response", state['draft_response'])},
            goto = "send_reply"
        )
    else:
        # Rejection means human will handle directly
        return Command(update = {}, goto = END)

def send_reply(state: EmailAgentState) -> EmailAgentState:
    """Send the email response"""
    # Integrate with a email service
    print(f"Sending reply: {state['draft_response'][:60]}...")
    return {}

# Build the graph

In [ ]:
# Create the graph
builder = StateGraph(EmailAgentState)

# Add nodes
builder.add_node("read_email", read_email)
builder.add_node("classify_intent", classify_intent)
builder.add_node("search_documentation", search_documentation)
builder.add_node("bug_tracking", bug_tracking)
builder.add_node("write_response", write_response)
builder.add_node("human_review", human_review)
builder.add_node("send_reply", send_reply)

# Add edges
builder.add_edge(START, "read_email")
builder.add_edge("read_email", "classify_intent")
builder.add_edge("classify_intent", "search_documentation")
builder.add_edge("classify_intent", "bug_tracking")
builder.add_edge("search_documentation", "write_response")
builder.add_edge("bug_tracking", "write_response")
builder.add_edge("send_reply", END)

# Compile with checkpointer for persistence
from langgraph.checkpoint.memory import InMemorySaver
memory = InMemorySaver()
app = builder.compile(checkpointer = memory)

In [ ]:
display(Image(app.get_graph().draw_mermaid_png()))

# Test

In [ ]:
# # Test with urgent billing issue
# initial_state = {
#     "email_content": "I was charged twice for my subscription! This is urgent!",
#     "sender_email": "customer@example.com",
#     "email_id": "email_123"
# }

# # Run with a thread_id for persistence
# config = {"configurable": {"thread_id": "customer_123"}}
# result = app.invoke(initial_state, config)

In [ ]:
# print(result.keys())

In [ ]:
# # The graph will pause at human_review
# print(f"Draft ready for review: {result['draft_response'][:60]}...\n")

# # # Provide human input to resume
# human_response = Command(
#     resume = {
#         "approved": True
#     }
# )

# # # Resume execution
# final_result = app.invoke(human_response, config)
# final_result = app.invoke(human_response, config)
# final_result = app.invoke(human_response, config)
# print(final_result['draft_response'])
# print("Email sent successfully!")

In [ ]:
# email_content = [
#     "I was charged two times for my subscription! This is urgent!",
#     "I was wondering if this was available in blue?",
#     "Can you tell me how long the sale is on?",
#     "The tire won't stay on the car!",
#     "My subscription is going to end in a few months, what is the new rate?"
# ]
# needs_approval = []

# for i, content in enumerate(email_content): 

#     initial_state = {
#         "email_content": content,
#         "sender_email": "customer@example.com",
#         "email_id": f"email_{i}",
#     }
#     print(f"{initial_state['email_id']}: ", end="")

#     thread_id = uuid.uuid4()
#     config =  {"configurable": {"thread_id": thread_id}}
#     result = app.invoke(initial_state, config)
#     if "__interrupt__" in result.keys():
#         result['thread_id'] = thread_id
#         needs_approval.append(result)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import uuid

# Clear cell output
clear_output(wait=True)

# Global state to track current workflow
current_state = {
    'thread_id': None,
    'config': None,
    'result': None
}

# Create widgets
email_content_input = widgets.Textarea(
    value='I was charged twice for my subscription! This is urgent!',
    placeholder='Enter customer email content...',
    description='Email:',
    continuous_update=False,
    layout=widgets.Layout(width='700px', height='100px')
)

sender_email_input = widgets.Text(
    value='customer@example.com',
    placeholder='customer@example.com',
    description='From:',
    layout=widgets.Layout(width='400px')
)

submit_btn = widgets.Button(
    description='Process Email',
    button_style='primary',
    icon='envelope',
    layout=widgets.Layout(width='150px')
)

# Review widgets (hidden initially)
review_section = widgets.VBox([], layout=widgets.Layout(display='none'))
draft_display = widgets.HTML(value='')
edit_draft = widgets.Textarea(
    placeholder='Edit the draft response...',
    layout=widgets.Layout(width='700px', height='150px')
)
approve_btn = widgets.Button(
    description='Approve & Send',
    button_style='success',
    icon='check',
    layout=widgets.Layout(width='150px')
)
reject_btn = widgets.Button(
    description='Reject',
    button_style='danger',
    icon='times',
    layout=widgets.Layout(width='150px')
)

# Result display
result_html = widgets.HTML(value='')

def on_submit_click(btn):
    """Handle email submission"""
    email_content = email_content_input.value.strip()
    sender_email = sender_email_input.value.strip()

    if not email_content:
        result_html.value = "<p style='color: red;'>⚠️ Please enter email content!</p>"
        return

    # Show processing state
    btn.disabled = True
    result_html.value = "<p>🔄 Processing email...</p>"
    review_section.layout.display = 'none'

    try:
        # Create initial state
        initial_state = {
            "email_content": email_content,
            "sender_email": sender_email,
            "email_id": f"email_{uuid.uuid4().hex[:8]}"
        }

        # Generate unique thread ID
        thread_id = str(uuid.uuid4())
        config = {"configurable": {"thread_id": thread_id}}

        # Store in global state
        current_state['thread_id'] = thread_id
        current_state['config'] = config

        # Invoke workflow
        result = app.invoke(initial_state, config)
        current_state['result'] = result

        # Check if interrupted for human review
        if "__interrupt__" in result:
            show_review_section(result)
        else:
            # Email was processed without human review
            show_success_result(result)

    except Exception as e:
        import traceback
        result_html.value = f"<p style='color: red;'>❌ Error: {str(e)}<br><pre>{traceback.format_exc()}</pre></p>"
    finally:
        btn.disabled = False

def show_review_section(result):
    """Display the review section when workflow is interrupted"""
    classification = result.get('classification', {})
    
    # Debug: Print the result structure
    print("DEBUG - Result keys:", result.keys())
    print("DEBUG - Draft in result:", result.get('draft_response', 'NOT FOUND'))
    
    # Access interrupt data correctly
    interrupts = result.get('__interrupt__', [])
    interrupt_data = {}
    
    if interrupts:
        print(f"DEBUG - Number of interrupts: {len(interrupts)}")
        interrupt_obj = interrupts[0]
        print(f"DEBUG - Interrupt object type: {type(interrupt_obj)}")
        
        # Access the value from the interrupt object
        if hasattr(interrupt_obj, 'value'):
            interrupt_data = interrupt_obj.value
            print("DEBUG - Interrupt data from .value:", interrupt_data)
        elif isinstance(interrupt_obj, tuple) and len(interrupt_obj) > 1:
            # It might be a tuple (namespace, interrupt_obj)
            if hasattr(interrupt_obj[1], 'value'):
                interrupt_data = interrupt_obj[1].value
                print("DEBUG - Interrupt data from tuple[1].value:", interrupt_data)
    
    # Try multiple ways to get the draft
    draft = (
        interrupt_data.get('draft_response') or 
        result.get('draft_response') or 
        ''
    )
    
    print(f"DEBUG - Final draft value: '{draft[:100] if draft else 'EMPTY'}'")
    
    original_email = interrupt_data.get('original_email', result.get('email_content', ''))
    urgency = interrupt_data.get('urgency', classification.get('urgency', 'unknown'))
    intent = interrupt_data.get('intent', classification.get('intent', 'unknown'))

    # Update draft editor with the draft content
    edit_draft.value = draft if draft else "No draft found - please check the debug output above"

    # Create urgency badge
    urgency_colors = {
        'critical': '#dc3545',
        'high': '#fd7e14',
        'medium': '#ffc107',
        'low': '#28a745'
    }
    urgency_color = urgency_colors.get(urgency, '#6c757d')

    # Display classification and draft
    draft_display.value = f"""
    <div style="font-family: 'Segoe UI', Arial, sans-serif; padding: 20px; background: #f8f9fa; border-radius: 8px;">
        <h3 style="color: #0066cc; margin-top: 0;">📧 Human Review Required</h3>
        <hr style="border: 1px solid #dee2e6;">

        <div style="margin: 15px 0;">
            <h4 style="color: #495057; margin-bottom: 10px;">Original Email</h4>
            <div style="background: white; padding: 15px; border-left: 4px solid #0066cc; border-radius: 4px;">
                <p style="margin: 0; color: #333; white-space: pre-wrap;">{original_email}</p>
            </div>
        </div>

        <div style="margin: 15px 0;">
            <h4 style="color: #495057; margin-bottom: 10px;">Classification</h4>
            <div style="background: white; padding: 15px; border-radius: 4px;">
                <p style="margin: 5px 0;"><strong>Intent:</strong> <span style="background: #e7f3ff; padding: 3px 8px; border-radius: 3px;">{intent}</span></p>
                <p style="margin: 5px 0;"><strong>Urgency:</strong> <span style="background: {urgency_color}; color: white; padding: 3px 8px; border-radius: 3px;">{urgency}</span></p>
            </div>
        </div>

        <div style="margin: 15px 0;">
            <h4 style="color: #495057; margin-bottom: 10px;">Draft Response (Edit below if needed)</h4>
        </div>
    </div>
    """

    # Show review section
    review_section.children = [
        draft_display,
        edit_draft,
        widgets.HBox([approve_btn, reject_btn], layout=widgets.Layout(margin='10px 0'))
    ]
    review_section.layout.display = 'flex'
    result_html.value = ''

def show_success_result(result):
    """Display success message"""
    classification = result.get('classification', {})
    draft = result.get('draft_response', '')

    result_html.value = f"""
    <div style="font-family: 'Segoe UI', Arial, sans-serif; padding: 20px; background: #d4edda; border-radius: 8px; border-left: 4px solid #28a745;">
        <h3 style="color: #155724; margin-top: 0;">✅ Email Processed Successfully</h3>
        <p style="margin: 5px 0;"><strong>Intent:</strong> {classification.get('intent', 'N/A')}</p>
        <p style="margin: 5px 0;"><strong>Urgency:</strong> {classification.get('urgency', 'N/A')}</p>
        <hr style="border: 1px solid #c3e6cb;">
        <h4 style="color: #155724;">Sent Response:</h4>
        <div style="background: white; padding: 15px; border-radius: 4px; white-space: pre-wrap; color: #333;">
{draft}
        </div>
    </div>
    """

def on_approve_click(btn):
    """Handle approval of draft"""
    btn.disabled = True
    approve_btn.disabled = True
    reject_btn.disabled = True

    try:
        edited_response = edit_draft.value

        # Resume workflow with approval
        human_response = Command(
            resume={
                "approved": True,
                "edited_response": edited_response
            }
        )

        # Continue execution
        final_result = app.invoke(human_response, current_state['config'])

        # Show success
        result_html.value = f"""
        <div style="font-family: 'Segoe UI', Arial, sans-serif; padding: 20px; background: #d4edda; border-radius: 8px; border-left: 4px solid #28a745;">
            <h3 style="color: #155724; margin-top: 0;">✅ Email Sent Successfully!</h3>
            <hr style="border: 1px solid #c3e6cb;">
            <h4 style="color: #155724;">Final Response:</h4>
            <div style="background: white; padding: 15px; border-radius: 4px; white-space: pre-wrap; color: #333;">
{edited_response}
            </div>
        </div>
        """

        # Hide review section
        review_section.layout.display = 'none'

    except Exception as e:
        import traceback
        result_html.value = f"<p style='color: red;'>❌ Error: {str(e)}<br><pre>{traceback.format_exc()}</pre></p>"
    finally:
        btn.disabled = False
        approve_btn.disabled = False
        reject_btn.disabled = False

def on_reject_click(btn):
    """Handle rejection of draft"""
    btn.disabled = True
    approve_btn.disabled = True
    reject_btn.disabled = True

    try:
        # Resume workflow with rejection
        human_response = Command(
            resume={
                "approved": False
            }
        )

        # Continue execution (will end)
        app.invoke(human_response, current_state['config'])

        # Show rejection message
        result_html.value = """
        <div style="font-family: 'Segoe UI', Arial, sans-serif; padding: 20px; background: #f8d7da; border-radius: 8px; border-left: 4px solid #dc3545;">
            <h3 style="color: #721c24; margin-top: 0;">❌ Draft Rejected</h3>
            <p style="color: #721c24; margin: 0;">The email will be handled manually by a human agent.</p>
        </div>
        """

        # Hide review section
        review_section.layout.display = 'none'

    except Exception as e:
        import traceback
        result_html.value = f"<p style='color: red;'>❌ Error: {str(e)}<br><pre>{traceback.format_exc()}</pre></p>"
    finally:
        btn.disabled = False
        approve_btn.disabled = False
        reject_btn.disabled = False

# Attach event handlers
submit_btn.on_click(on_submit_click)
approve_btn.on_click(on_approve_click)
reject_btn.on_click(on_reject_click)

# Display UI
display(widgets.VBox([
    widgets.HTML("""
        <h2 style='color: #0066cc;'>📧 Email Agent - Human-in-the-Loop</h2>
        <div style="background: #e7f3ff; padding: 15px; border-radius: 5px; margin-bottom: 15px; border-left: 4px solid #0066cc;">
            <h4 style="margin-top: 0;">How This Works:</h4>
            <ol style="margin-bottom: 0;">
                <li><strong>Email Classification:</strong> AI analyzes the email intent and urgency</li>
                <li><strong>Context Gathering:</strong> Searches documentation or creates bug tickets as needed</li>
                <li><strong>Draft Generation:</strong> AI creates an appropriate response</li>
                <li><strong>Human Review:</strong> High-urgency or complex emails pause for your approval</li>
                <li><strong>Send:</strong> Approved responses are sent automatically</li>
            </ol>
            <p style="margin: 10px 0 0 0;"><strong>Try these examples:</strong></p>
            <ul style="margin-top: 5px; margin-bottom: 0;">
                <li>"I was charged twice for my subscription! This is urgent!"</li>
                <li>"I was wondering if this was available in blue?"</li>
                <li>"The app crashes when I click the login button"</li>
                <li>"How do I reset my password?"</li>
            </ul>
        </div>
    """),
    widgets.HBox([sender_email_input]),
    email_content_input,
    widgets.HBox([submit_btn]),
    review_section,
    result_html
], layout=widgets.Layout(padding='20px')))

>LangSmith Trace - [Start-to-End](https://smith.langchain.com/public/3898d0d0-c934-4681-b325-7c4e1e88a826/r)  
>LangSmith Trace - [Interrupt](https://smith.langchain.com/public/c23a3aed-cfa8-42aa-8f1e-78f58941aecd/r)